# Test creation

This notebook will create the necessary folder structure for later use. See examples below.

In [ ]:
%load_ext autoreload
%autoreload 2

## Test information

In [ ]:
from src.data.test import *
import sys
from os.path import join

In [ ]:
#----
# Is you are creating a new test (True) or if the test is going to be updated (False)
isnewtest = True
# Date when you performed the test in 'YYYY-MM-DD' format
date = '2020-01-24'
# INTernal or EXTernal test (made by your organisation or others)
who = 'INT' 
# Short title for the test
short_name = 'AMAZING_TEST'
# Comment for your test. Make this as long as you need to describe fully the purpose of this test
comment = '''
'''
# Project
project = 'SmartCitizen'
# Firmware version
commit = ''
# Who made the test
author = 'Myself'
# Test type (indoor, outdoor, anything that helps you organise later on)
type_test = 'outdoor'
# If you are going to document the results somewhere, you can put a link below
report = ''
notes = ''
#----

# Create test object
newtest = test_wrapper(pd.to_datetime(date).strftime('%Y-%m') + '_' + who + '_' + short_name)

# Add General test details
details = {'project': project, 
           'commit': commit, 
           'author': author, 
           'type_test': type_test, 
           'report': report, 
           'comment': comment,
           'notes': notes}

# Output
print (newtest.name)

## Add devices

### Manual input

In [ ]:
STATIONS = {'10752':{'CO':162700911,'NO2':202702041,'OX':204702949},
            '10751':{'CO':162700906,'NO2':202702038,'OX':204702950}}

list_devices = list()

for device_name in STATIONS.keys():    
    list_devices.append(device_wrapper({'device_id': device_name,
                                         'type': 'STATION', 
                                         'version': '2.1', 
                                         'alphasense': {'CO': STATIONS[device_name]['CO'],
                                                        'NO2': STATIONS[device_name]['NO2'],
                                                        'O3': STATIONS[device_name]['OX'],
                                                        'slots': ['CO', 'NO2', 'O3']
                                                       },
                                         'pm_sensor': 'PMS5003', 
                                         'location': 'Europe/Madrid',
                                         'frequency': '1Min',
                                         'source': 'api'}, newtest.data))

## Process the test

In [ ]:
# Process the stuff
newtest.create(details, list_devices)

## Examples

In [ ]:
# ----------------------------------
# Example of csv device with new csv
# ----------------------------------
# device_wrapper({'name': device_name,
#                 'type': 'KIT', 
#                 'version': '2.1', 
#                 'pm_sensor': 'PMS5003', 
#                 'location': 'Europe/London',
#                 'frequency': '1Min',
#                 'source': 'csv_new',
#                 'fileNameRaw': device_name + '.csv', 
#                 'fileNameInfo': ''})
# ----------------------------------
# Example of device from API with alphasense data
# ----------------------------------
# device_wrapper({'device_id': device_name,
#                 'type': 'STATION', 
#                 'version': '2.1', 
#                 'alphasense': {'CO': 162581708,
#                                'NO2': 202160421,
#                                'O3': 204160144,
#                                'slots': ['CO', 'NO2', 'O3']
#                               },
#                 'pm_sensor': 'PMS5003', 
#                 'location': 'Europe/London',
#                 'frequency': '1Min',
#                 'source': 'api'})
# ----------------------------------
# Example of device from API in history
# ----------------------------------
# device_wrapper({'device_id': 5262,
#                 'type': 'STATION', 
#                 'version': '2.1', 
#                 'device_history': '5262', 
#                 'pm_sensor': 'PMS5003', 
#                 'location': 'Europe/London',
#                 'frequency': '1Min',
#                 'source': 'api'})
# ----------------------------------
# Example of reference device
# ----------------------------------
# device_wrapper({'name': 'PARROT_3',
#                 'type': 'REFERENCE', 
#                 'location': 'Europe/London',
#                 'fileNameRaw': 'grow_parrot_u6dODj2Dnm1570629407808.csv', 
#                 'fileNameInfo': '',
#                 'source': 'csv',
#                 'equipment': 'PARROT_SENSOR',
#                 'index': {'name' : 'Time','format' : '%Y-%m-%d %H:%M:%S', 'frequency' : '15Min'}, 
#                 'channels': {'pollutants' : ('air_temperature_celsius', 'battery_percent', 'calibrated_soil_moisture_percent', 'fertilizer_level', 'light', 'soil_moisture_percent', 'water_tank_level_percent'), 
#                            'units' : ('degC', '%', '%', '-', 'lux', '%', '%'),
#                            'names' : ('TEMP', 'BATT', 'Cal Soil Moisture', 'Fertilizer', 'Soil Moisture', 'Water Level')
#                               },
#                 'location': 'Europe/Madrid'})
# ----------------------------------
# Example of MUV_STATION
# ----------------------------------
# device_wrapper({'name': device_name,
#                  'type': 'OTHER', 
#                  'location': 'Europe/Madrid',
#                  'fileNameRaw': device_name + '.csv', 
#                  'fileNameInfo': '',
#                  'source': 'csv',
#                  'equipment': 'MUV_STATION',
#                  'index': {'name' : 'Time','format' : '%Y-%m-%d %H:%M:%S', 'frequency' : '3Min'}, 
#                  'channels': {'source_channel_names' : ('NOISE_A','HUM','GB_2W','GB_2A','GB_3W','GB_3A','EXT_PM_10','EXT_PM_25','TEMP'), 
#                             'units' : ('dB','%','mV','mV','mV','mV','ug/m3','ug/m3','ug/m3','degC'),
#                             'target_channel_names' : ('NOISE_A','HUM','GB_2W','GB_2A','GB_3W','GB_3A','EXT_PM_10','EXT_PM_25','TEMP')
#                                },
#                  'location': 'Europe/Madrid'})